In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

Using TensorFlow backend.


In [ ]:
bayes_iters = 3
batch_size = 256
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 32, 32

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
input_shape = (img_rows, img_cols, 3)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
def get_dropout(input_tensor, p=0.5, mc=False):
    if mc:
        return Dropout(p)(input_tensor, training=True)
    else:
        return Dropout(p)(input_tensor, training=False)


def get_model(mc=False, p=0.5, act="relu"):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation=act)(inp)
    x = Conv2D(64, kernel_size=(3, 3), activation=act)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation=act)(x)
    x = get_dropout(x, p=p, mc=mc)
    out = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [7]:
model = get_model(mc=False, act="relu")


In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0   

In [9]:
h = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 179s 4ms/step - loss: 1.6766 - accuracy: 0.3992 - val_loss: 1.3359 - val_accuracy: 0.5196


In [10]:
model.save_weights('initial_model_weights.h5')

In [15]:
dropout_post_predictions = []
    
for i in range(bayes_iters):
    print("Iteration: ", i+1)
    #Generate random p value for Dropout ratio
    p_rand = np.random.random()
    print("Random p: ", np.round(p_rand,2))
    drop_model = get_model(mc=True, p= p_rand, act="relu")
    
    #Initialize model with pre-trained weights
    drop_model.load_weights('initial_model_weights.h5')
    h_mc = drop_model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    # Accuracy of model
    score = drop_model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    #Posterior predictions
    y_pred = drop_model.predict(x_test)
    dropout_post_predictions.append(y_pred)


Iteration:  1
Random p:  0.2
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 183s 4ms/step - loss: 1.2400 - accuracy: 0.5640 - val_loss: 1.1169 - val_accuracy: 0.6099
Test loss: 1.1171069507598876
Test accuracy: 0.6122999787330627
Iteration:  2
Random p:  0.51
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 184s 4ms/step - loss: 1.3636 - accuracy: 0.5220 - val_loss: 1.2814 - val_accuracy: 0.5527
Test loss: 1.285919616317749
Test accuracy: 0.5516999959945679
Iteration:  3
Random p:  0.32
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 188s 4ms/step - loss: 1.2799 - accuracy: 0.5461 - val_loss: 1.2004 - val_accuracy: 0.5826
Test loss: 1.2031252590179444
Test accuracy: 0.5828999876976013


In [19]:
np.array(dropout_post_predictions).shape

(3, 10000, 10)

In [21]:
posterior_mean_preds = np.mean(np.array(dropout_post_predictions),axis=0)
posterior_mean_preds.shape

(10000, 10)